In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

In [3]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [4]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [6]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

In [7]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='727326e7-cf92-4dc1-8968-9817e22d3c45'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'The user says "My favourite colour is green". We need to update the favourite colour in the state using the provided function update_favourite_colour. So we call that function with favourite_colour: "green".', 'tool_calls': [{'id': 'fc_54fa30c5-849d-432c-83a5-16658c7ba4fb', 'function': {'arguments': '{"favourite_colour":"green"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 137, 'total_tokens': 210, 'completion_time': 0.150940649, 'completion_tokens_details': {'reasoning_tokens': 42}, 'prompt_time': 0.005756704, 'prompt_tokens_details': None, 'queue_time': 0.005382443, 'total_time': 0.156697353}, 'model_name': 'openai/gpt-oss-120b', 'syst

In [8]:
print(response["messages"][-1].content)

Got it—your favourite colour is now set to **green**! If there’s anything else you’d like to share or ask, just let me know.


In [9]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='5afa1d85-f685-44ba-9dfe-089955d3ae09'),
              AIMessage(content="Hello! I'm doing great, thanks for asking. How can I assist you today?", additional_kwargs={'reasoning_content': 'We need to respond. The user says hello. We can respond politely. No need to call function.'}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 138, 'total_tokens': 186, 'completion_time': 0.099448456, 'completion_tokens_details': {'reasoning_tokens': 22}, 'prompt_time': 0.005160486, 'prompt_tokens_details': None, 'queue_time': 0.004707694, 'total_time': 0.104608942}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_c9b3d7b164', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c530d-c8e5-7191-ad35-74a639a6e37b-0', tool_calls=[], invalid_tool_calls=[], usage_metada

In [10]:
print(response["messages"][-1].content)

Hello! I'm doing great, thanks for asking. How can I assist you today?


## Read state

In [11]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    model=model,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [12]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is blue")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'blue',
 'messages': [HumanMessage(content='My favourite colour is blue', additional_kwargs={}, response_metadata={}, id='1564878b-59a8-4e2d-9819-05a99cc2247a'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'The user says "My favourite colour is blue". We need to store this in state using update_favourite_colour. Then maybe respond acknowledging. Use function.', 'tool_calls': [{'id': 'fc_13a8a0ed-9f62-43fc-b5ac-611513540ae0', 'function': {'arguments': '{"favourite_colour":"blue"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 161, 'total_tokens': 224, 'completion_time': 0.132739535, 'completion_tokens_details': {'reasoning_tokens': 32}, 'prompt_time': 0.006082393, 'prompt_tokens_details': None, 'queue_time': 0.004717608, 'total_time': 0.138821928}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_c9b3d7b164', 'service_tier': 'on_de

In [13]:
print(response["messages"][-1].content)

Got it! I’ve noted that your favourite colour is **blue**. If there’s anything else you’d like to share or ask, just let me know!


In [14]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'blue',
 'messages': [HumanMessage(content='My favourite colour is blue', additional_kwargs={}, response_metadata={}, id='1564878b-59a8-4e2d-9819-05a99cc2247a'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'The user says "My favourite colour is blue". We need to store this in state using update_favourite_colour. Then maybe respond acknowledging. Use function.', 'tool_calls': [{'id': 'fc_13a8a0ed-9f62-43fc-b5ac-611513540ae0', 'function': {'arguments': '{"favourite_colour":"blue"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 161, 'total_tokens': 224, 'completion_time': 0.132739535, 'completion_tokens_details': {'reasoning_tokens': 32}, 'prompt_time': 0.006082393, 'prompt_tokens_details': None, 'queue_time': 0.004717608, 'total_time': 0.138821928}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_c9b3d7b164', 'service_tier': 'on_de

In [15]:
print(response["messages"][-1].content)

Your favourite colour is **blue**.
